In [ ]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [ ]:
import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

In [1]:
# 1000 genomes obejrzeć w BAM - wariant o call rate niskim - obejrzeć w próbce
# czy warianty bez calli są wszędzie czy gdzieniegdzie?

# rozwiązanie 1 - call rate jest niższe (92%) tam gdzie co najmniej jedna próbka europejska jest non-ref

# ogólnie to się dziwnie stało, bo się przefiltrowały entries w którymś momencie, a nie wiem czemu??

In [24]:
from hail.ggplot import *

import plotly
import plotly.io as pio
pio.renderers.default='iframe'

In [12]:
# first filter per interval, then densify then calcuate coverage and then 

mt_paths = [
    '/net/ascratch/people/plggosborcz/1kg.vds',
    '/net/ascratch/people/plggosborcz/1kg-eur.vds',
    '/net/ascratch/people/plggosborcz/1kg-eur-to-merge.vds',
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/full.vds', #this is all before interval filtering
    '/net/ascratch/people/plggosborcz/full-protein-coding.vds', #this is after interval filtering
    '/net/ascratch/people/plggosborcz/full-rpkm.vds'
    '/net/ascratch/people/plggosborcz/gts.vds',
]

to_filter = hl.parse_locus_interval('chr1:10000000-15000000', reference_genome='GRCh38') # 5mb

In [13]:
vdses = []
mts = []

for file in mt_paths:
    
    full = hl.vds.read_vds(file)
    full = hl.vds.filter_intervals(full, [to_filter], split_reference_blocks=True)
    full.write(localfs_path+file.split('/')[-1]+'-filtered-vds')
               
    full = hl.vds.read_vds(localfs_path+file.split('/')[-1]+'-filtered-vds')
    
    vdses.append(full)
    
    vd_gt = full.variant_data.transmute_entries(
    GT = hl.vds.lgt_to_gt(full.variant_data.LGT, full.variant_data.LA))
    mt = hl.vds.to_dense_mt(hl.vds.VariantDataset(full.reference_data, vd_gt))
    
    mt = mt.naive_coalesce(200)
    
    mt = hl.sample_qc(mt)

    
    mt = mt.annotate_cols(
        group = hl.if_else(
            mt.s.contains('B'),
            'local_controls',
            hl.if_else(
                (mt.s.contains('NA') | mt.s.contains('HG')),
                '1kg_controls',
                'GTS'
            )
        )
    )  
    
    mt = mt.checkpoint(localfs_path+file.split('/')[-1]+'-qced')
    
    mts.append(mt)

# this already leads to mts that have different numbers of rows, so I guess to_dense_mt

2023-04-09 15:05:52.450 Hail: WARN: 'hl.vds.filter_intervals': filtering intervals when reference blocks have not been truncated
  (by 'hl.vds.truncate_reference_blocks') requires a full pass over the reference data (expensive!)
2023-04-09 15:05:55.652 Hail: INFO: Coerced sorted dataset
2023-04-09 15:05:56.722 Hail: INFO: Coerced sorted dataset
2023-04-09 15:05:57.731 Hail: INFO: Coerced sorted dataset
2023-04-09 15:53:24.377 Hail: INFO: wrote matrix table with 4863647 rows and 288 columns in 119993 partitions to /localfs/2214337/gts.vds-filtered-vds/reference_data
2023-04-09 15:53:29.699 Hail: INFO: wrote matrix table with 184054 rows and 288 columns in 207 partitions to /localfs/2214337/gts.vds-filtered-vds/variant_data
2023-04-09 16:07:57.754 Hail: INFO: wrote matrix table with 184054 rows and 288 columns in 200 partitions to /localfs/2214337/gts.vds-qced


In [55]:
test = hl.read_matrix_table('/localfs/2214337/1kg-eur-to-merge.vds-filtered-vds')

FatalError: FileNotFoundException: File /localfs/2214337/1kg-eur-to-merge.vds-filtered-vds/metadata.json.gz does not exist

Java stack trace:
java.io.FileNotFoundException: File /localfs/2214337/1kg-eur-to-merge.vds-filtered-vds/metadata.json.gz does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:666)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:987)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:656)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:454)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:146)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:347)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:899)
	at is.hail.io.fs.HadoopFS.openNoCompression(HadoopFS.scala:92)
	at is.hail.io.fs.FS.open(FS.scala:360)
	at is.hail.io.fs.FS.open$

In [50]:
vdses[2].variant_data.count()

IndexError: list index out of range

In [18]:
vdses[0].reference_data.count()

(2407554, 4151)

In [ ]:
kg_vds = vdses[0]

#make the 1kg table consistent with the other table:
kg_vds.reference_data = kg_vds.reference_data.select_entries(
    END = kg_vds.reference_data.END,
    DP = kg_vds.reference_data.DP,
    GQ = kg_vds.reference_data.GQ,
    MIN_DP = kg_vds.reference_data.MIN_DP
)

kg_vds.variant_data = kg_vds.variant_data.select_entries(
    LA = kg_vds.variant_data.LA,
    LGT = kg_vds.variant_data.LGT,
    LAD = kg_vds.variant_data.LAD,
    LPGT = kg_vds.variant_data.LPGT,
    LPL = kg_vds.variant_data.LPL,
    RGQ = kg_vds.variant_data.RGQ,
    gvcf_info = kg_vds.variant_data.gvcf_info.select(
            BaseQRankSum = kg_vds.variant_data.gvcf_info.BaseQRankSum,
            ExcessHet = hl.missing(hl.tfloat),
            InbreedingCoeff = hl.missing(hl.tfloat),
            MLEAC = hl.literal([0]),
            MLEAF = hl.literal([0.1]),
            MQRankSum = kg_vds.variant_data.gvcf_info.MQRankSum,
            RAW_MQandDP = hl.literal([0]),
            ReadPosRankSum = kg_vds.variant_data.gvcf_info.ReadPosRankSum
        ),
    DP = kg_vds.variant_data.DP,
    GQ = kg_vds.variant_data.GQ,
    MIN_DP = kg_vds.variant_data.MIN_DP,
    PID = kg_vds.variant_data.PID,
    PS = hl.missing(hl.tint),
    SB = kg_vds.variant_data.SB
)

kg_vds.write(localfs_path+'1kg-all-to-merge.vds')

In [23]:
vdses[1].variant_data.count()

(77421, 501)

In [24]:
vdses[1].reference_data.count()

(920934, 501)

In [26]:
kg_vds = vdses[1]

#make the 1kg table consistent with the other table:
kg_vds.reference_data = kg_vds.reference_data.select_entries(
    END = kg_vds.reference_data.END,
    DP = kg_vds.reference_data.DP,
    GQ = kg_vds.reference_data.GQ,
    MIN_DP = kg_vds.reference_data.MIN_DP
)

kg_vds.variant_data = kg_vds.variant_data.select_entries(
    LA = kg_vds.variant_data.LA,
    LGT = kg_vds.variant_data.LGT,
    LAD = kg_vds.variant_data.LAD,
    LPGT = kg_vds.variant_data.LPGT,
    LPL = kg_vds.variant_data.LPL,
    RGQ = kg_vds.variant_data.RGQ,
    gvcf_info = kg_vds.variant_data.gvcf_info.select(
            BaseQRankSum = kg_vds.variant_data.gvcf_info.BaseQRankSum,
            ExcessHet = hl.missing(hl.tfloat),
            InbreedingCoeff = hl.missing(hl.tfloat),
            MLEAC = hl.literal([0]),
            MLEAF = hl.literal([0.1]),
            MQRankSum = kg_vds.variant_data.gvcf_info.MQRankSum,
            RAW_MQandDP = hl.literal([0]),
            ReadPosRankSum = kg_vds.variant_data.gvcf_info.ReadPosRankSum
        ),
    DP = kg_vds.variant_data.DP,
    GQ = kg_vds.variant_data.GQ,
    MIN_DP = kg_vds.variant_data.MIN_DP,
    PID = kg_vds.variant_data.PID,
    PS = hl.missing(hl.tint),
    SB = kg_vds.variant_data.SB
)

kg_vds.write(localfs_path+'1kg-eur-to-merge.vds')

2023-04-09 17:15:45.780 Hail: INFO: wrote matrix table with 920934 rows and 501 columns in 50001 partitions to /localfs/2214337/1kg-eur-to-merge.vds/reference_data
2023-04-09 17:15:48.345 Hail: INFO: wrote matrix table with 77421 rows and 501 columns in 87 partitions to /localfs/2214337/1kg-eur-to-merge.vds/variant_data


now I have two joined vdses - gts with european samples and gts without european samples and I will export them to dense mts, perform matrix qc and see what happens - does it matter that I filtered the samples first (in my mind it shouldn't

### repartition the vds's

In [3]:
kg_all = hl.vds.read_vds(localfs_path+'1kg-all-to-merge.vds')
gts = hl.vds.read_vds(localfs_path+'gts.vds-filtered-vds')
kg_eur = hl.vds.read_vds(localfs_path+'1kg-eur-to-merge.vds')

In [8]:
hl.vds.VariantDataset(
    kg_all.reference_data.naive_coalesce(100),
    kg_all.variant_data.naive_coalesce(100)).write(localfs_path+'1kg-all-to-merge-s.vds')

hl.vds.VariantDataset(
    gts.reference_data.naive_coalesce(100),
    gts.variant_data.naive_coalesce(100)).write(localfs_path+'gts.vds-filtered-s.vds')

In [11]:
hl.vds.VariantDataset(
    kg_eur.reference_data.naive_coalesce(100),
    kg_eur.variant_data.naive_coalesce(100)).write(localfs_path+'1kg-eur-to-merge-s-2.vds')

2023-04-10 11:45:07.908 Hail: INFO: wrote matrix table with 920934 rows and 501 columns in 100 partitions to /localfs/2214337/1kg-eur-to-merge-s-2.vds/reference_data
2023-04-10 11:45:10.339 Hail: INFO: wrote matrix table with 77421 rows and 501 columns in 87 partitions to /localfs/2214337/1kg-eur-to-merge-s-2.vds/variant_data


In [10]:
#this is with all samples

combiner_all = hl.vds.new_combiner(
    output_path = localfs_path+'gts_1kg_all_5.vds',
    temp_path = localfs_path+'gts_1kg_all_tmp_5',
    save_path = localfs_path+'gts_1kg_all_sv_5',
    vds_paths=[
        localfs_path+'1kg-all-to-merge-s.vds',
        localfs_path+'gts.vds-filtered-s.vds'         
    ],
    reference_genome = 'GRCh38',
    use_genome_default_intervals = True,
    gvcf_batch_size = 25,
    branch_factor=50
)

combiner_all.run()
combiner_all.save()

2023-04-10 11:41:03.296 Hail: INFO: Running VDS combiner:
    VDS arguments: 2 datasets with 4439 samples
    GVCF arguments: 0 inputs/samples
    Branch factor: 50
    GVCF merge batch size: 25
2023-04-10 11:41:03.399 Hail: INFO: VDS Combine (job 1): merging 2 datasets with 4439 samples
2023-04-10 11:41:55.407 Hail: INFO: wrote table with 2407554 rows in 100 partitions to /localfs/2214337/gts_1kg_all_tmp_5/combiner-intermediates/2f042d3b-6004-4acf-8fac-9b3ac1d3ef07_vds-combine_job1/interval_checkpoint.ht
2023-04-10 11:42:23.867 Hail: INFO: wrote matrix table with 4871863 rows and 4439 columns in 102 partitions to /localfs/2214337/gts_1kg_all_5.vds/reference_data
    Total size: 2.20 GiB
    * Rows/entries: 2.20 GiB
    * Columns: 17.19 KiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (305.00 B)
    * Largest partition:  83544 rows (30.00 MiB)
2023-04-10 11:43:21.402 Hail: INFO: wrote matrix table with 438460 rows and 4439 columns in 102 partitions to /localfs/2214337/gts_1k

In [12]:
combiner_all = hl.vds.new_combiner(
    output_path = localfs_path+'gts_1kg_eur.vds',
    temp_path = localfs_path+'gts_1kg_eur_tmp',
    save_path = localfs_path+'gts_1kg_eur_sv',
    vds_paths=[
        localfs_path+'1kg-eur-to-merge-s-2.vds',
        localfs_path+'gts.vds-filtered-s.vds'         
    ],
    reference_genome = 'GRCh38',
    use_genome_default_intervals = True,
    gvcf_batch_size = 25,
    branch_factor=50
)

combiner_all.run()
combiner_all.save()

2023-04-10 11:47:14.380 Hail: INFO: Running VDS combiner:
    VDS arguments: 2 datasets with 789 samples
    GVCF arguments: 0 inputs/samples
    Branch factor: 50
    GVCF merge batch size: 25
2023-04-10 11:47:14.451 Hail: INFO: VDS Combine (job 1): merging 2 datasets with 789 samples
2023-04-10 11:47:18.705 Hail: INFO: wrote table with 920934 rows in 100 partitions to /localfs/2214337/gts_1kg_eur_tmp/combiner-intermediates/7499b210-b5b6-4b7b-881c-106bc95794ec_vds-combine_job1/interval_checkpoint.ht
2023-04-10 11:47:40.131 Hail: INFO: wrote matrix table with 4867881 rows and 789 columns in 40 partitions to /localfs/2214337/gts_1kg_eur.vds/reference_data
    Total size: 1.56 GiB
    * Rows/entries: 1.56 GiB
    * Columns: 3.21 KiB
    * Globals: 11.00 B
    * Smallest partition: 17 rows (5.46 KiB)
    * Largest partition:  208272 rows (65.31 MiB)
2023-04-10 11:47:49.390 Hail: INFO: wrote matrix table with 216804 rows and 789 columns in 40 partitions to /localfs/2214337/gts_1kg_eur.vds/

In [13]:
# now this is gts + all
vdses = [
    localfs_path+'gts_1kg_all_5.vds',
    localfs_path+'gts_1kg_eur.vds'
]
# now this is gts + eur

In [16]:
for vds in vdses[1:]:
    
    full = hl.vds.read_vds(vds)
    
    vd_gt = full.variant_data.transmute_entries(
    GT = hl.vds.lgt_to_gt(full.variant_data.LGT, full.variant_data.LA))
    mt = hl.vds.to_dense_mt(hl.vds.VariantDataset(full.reference_data, vd_gt))
    
    mt = mt.naive_coalesce(50)
    mt = hl.sample_qc(mt)
    mt = hl.variant_qc(mt)

    
    mt = mt.annotate_cols(
        group = hl.if_else(
            mt.s.contains('B'),
            'local_controls',
            hl.if_else(
                (mt.s.contains('NA') | mt.s.contains('HG')),
                '1kg_controls',
                'GTS'
            )
        )
    )  
    
    mt = mt.checkpoint(localfs_path+vds.split('/')[-1]+'-qced-new-test')

2023-04-10 11:59:00.434 Hail: INFO: wrote matrix table with 216804 rows and 789 columns in 40 partitions to /localfs/2214337/gts_1kg_eur.vds-qced-new-test


In [18]:
qced = [
    hl.read_matrix_table('/localfs/2214337/gts_1kg_all_5.vds-qced-new-test'),
    hl.read_matrix_table('/localfs/2214337/gts_1kg_eur.vds-qced-new-test')
]   

In [39]:
joined = hl.read_matrix_table('/localfs/2214337/full.vds-qced')

In [40]:
joined.count()

(216805, 789)

In [44]:
joined.sample_qc.show()

+-----------+-------------------------+--------------------------+
| s         | sample_qc.dp_stats.mean | sample_qc.dp_stats.stdev |
+-----------+-------------------------+--------------------------+
| str       |                 float64 |                  float64 |
+-----------+-------------------------+--------------------------+
| "NA06985" |                3.21e+01 |                 7.72e+00 |
| "NA06986" |                3.03e+01 |                 7.26e+00 |
| "NA06994" |                2.90e+01 |                 7.94e+00 |
| "NA07000" |                3.43e+01 |                 8.38e+00 |
| "NA07037" |                3.21e+01 |                 7.89e+00 |
| "NA07051" |                3.04e+01 |                 7.61e+00 |
| "NA07347" |                3.11e+01 |                 7.57e+00 |
| "NA07357" |                2.86e+01 |                 7.06e+00 |
| "NA10847" |                3.63e+01 |                 1.58e+01 |
| "NA10851" |                3.05e+01 |                 8.06e+00 |
+-----------+-------------------------+--------------------------+

+------------------------+------------------------+-------------------------+
| sample_qc.dp_stats.min | sample_qc.dp_stats.max | sample_qc.gq_stats.mean |
+------------------------+------------------------+-------------------------+
|                float64 |                float64 |                 float64 |
+------------------------+------------------------+-------------------------+
|               0.00e+00 |               9.40e+01 |                4.64e+01 |
|               0.00e+00 |               9.60e+01 |                4.36e+01 |
|               0.00e+00 |               8.90e+01 |                4.16e+01 |
|               0.00e+00 |               1.07e+02 |                5.14e+01 |
|               0.00e+00 |               8.80e+01 |                4.74e+01 |
|               0.00e+00 |               8.70e+01 |                4.42e+01 |
|               0.00e+00 |               9.20e+01 |                4.46e+01 |
|               1.00e+00 |               7.80e+01 |                4.18e+01 |
|               0.00e+00 |               1.99e+02 |                5.10e+01 |
|               0.00e+00 |               9.10e+01 |                4.29e+01 |
+------------------------+------------------------+-------------------------+

+--------------------------+------------------------+------------------------+
| sample_qc.gq_stats.stdev | sample_qc.gq_stats.min | sample_qc.gq_stats.max |
+--------------------------+------------------------+------------------------+
|                  float64 |                float64 |                float64 |
+--------------------------+------------------------+------------------------+
|                 2.59e+01 |               0.00e+00 |               9.90e+01 |
|                 2.49e+01 |               0.00e+00 |               9.90e+01 |
|                 2.48e+01 |               0.00e+00 |               9.90e+01 |
|                 2.71e+01 |               0.00e+00 |               9.90e+01 |
|                 2.63e+01 |               0.00e+00 |               9.90e+01 |
|                 2.55e+01 |               0.00e+00 |               9.90e+01 |
|                 2.55e+01 |               0.00e+00 |               9.90e+01 |
|                 2.44e+01 |               0.00e+00 |               9.90e+01 |
|                 2.85e+01 |               0.00e+00 |               9.90e+01 |
|                 2.54e+01 |               0.00e+00 |               9.90e+01 |
+--------------------------+------------------------+------------------------+

+---------------------+--------------------+------------------------+
| sample_qc.call_rate | sample_qc.n_called | sample_qc.n_not_called |
+---------------------+--------------------+------------------------+
|             float64 |              int64 |                  int64 |
+---------------------+--------------------+------------------------+
|            2.87e-01 |              62220 |                   

In [49]:
joined.entries().count()

92333001

In [47]:
qced[1].entries().count()

163406552

In [42]:
qced[1].entries().count()

,,,,,
,,'NA06985','NA06986','NA06994','NA07000'
locus,alleles,GT,GT,GT,GT
locus<GRCh38>,array<str>,call,call,call,call
chr1:10000044,"[""A"",""T""]",0/0,0/0,0/0,0/0
chr1:10000052,"[""G"",""A""]",0/0,0/0,0/0,0/0
chr1:10000053,"[""C"",""A""]",0/0,0/0,0/0,0/0
chr1:10000054,"[""C"",""A""]",0/0,0/0,0/0,0/0
chr1:10000306,"[""T"",""G""]",0/0,0/0,0/0,0/0
chr1:10000323,"[""G"",""T""]",0/0,0/0,0/0,0/0
chr1:10000329,"[""T"",""A""]",0/0,0/0,0/0,0/0


In [34]:
ggplot(
    joined,
    aes(x=joined.sample_qc.call_rate, fill=joined.group)
) + geom_histogram(
    alpha=0.4,
    position = 'identity',
    bins = 100
)

In [19]:
eur = hl.import_table('/net/pr2/projects/plgrid/plggneuromol/resources/1kg/europeans', no_header=True)
eur = eur.select(eur.f0)
eur = eur.f0.collect()

2023-04-10 12:01:35.164 Hail: INFO: wrote table with 501 rows in 1 partition to /localfs/2214337/tmp_hail/persist_tablendDQT3Fug0
2023-04-10 12:01:35.769 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)


In [20]:
qced[0] = qced[0].annotate_cols(
        eur = hl.if_else(
            hl.literal(eur).contains(qced[0].s),
            'eur',
            'else'
        )
)

In [22]:
qced[1] = qced[1].annotate_cols(
        eur = hl.if_else(
            hl.literal(eur).contains(qced[1].s),
            'eur',
            'else'
        )
)

In [25]:
ggplot(
    qced[0],
    aes(x=qced[0].sample_qc.call_rate, fill=qced[0].eur)
) + geom_histogram(
    alpha=0.4,
    position = 'identity',
    bins = 100
)

In [26]:
ggplot(
    qced[0],
    aes(x=qced[0].sample_qc.call_rate, fill=qced[0].group)
) + geom_histogram(
    alpha=0.4,
    position = 'identity',
    bins = 100
)

In [27]:
ggplot(
    qced[1],
    aes(x=qced[1].sample_qc.call_rate, fill=qced[1].eur)
) + geom_histogram(
    alpha=0.4,
    position = 'identity',
    bins = 100
)

In [30]:
ggplot(
    qced[1],
    aes(x=qced[1].sample_qc.call_rate, fill=qced[1].group)
) + geom_histogram(
    alpha=0.4,
    position = 'identity',
    bins = 100
)

In [37]:
ggplot(
    qced[1],
    aes(x=qced[1].sample_qc.call_rate, fill=qced[1].group)
) + geom_histogram(
    alpha=0.4,
    position = 'identity',
    bins = 20
)

In [ ]:
# now perform test filtering and update samples qc:
test_filter = qced[0].annotate_rows(
    eur_non_ref = hl.agg.filter(
        qced[0].eur == 'eur',
        hl.agg.any(qced[0].GT.is_non_ref())
    )
)

In [ ]:
test_filter = test_filter.filter_rows(
    test_filter.eur_non_ref
)

In [ ]:
test_filter = hl.sample_qc(test_filter)
test_filter = test_filter.checkpoint(localfs_path+'test-filter-qced-2')

In [ ]:
test_filter.count()

In [ ]:
qced[0].count()

In [ ]:
qced[1].count()